In [1]:
require 'rnn'

true	


In [2]:
function build_network(vocab_size, embed_dim)
    model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- returns a sequence-length x batch-size x embedDim tensor
    :add(nn.Sum(1, embed_dim, true)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.Tanh())                     ---     :add(nn.ReLU()) <- this one did worse
   return model
end

In [3]:
x = torch.LongTensor{{0, 1, 2 ,3, 4}, {0, 0, 1, 4, 3}}

In [4]:
nnm = build_network(4, 10)

In [5]:
nnm:forward(x:t())

 0.0174  0.0265 -0.1396 -0.2375  0.1837 -0.1387  0.0207 -0.1156  0.2890  0.0694
-0.0713  0.0036 -0.1932 -0.2810  0.1266  0.0023 -0.1511 -0.1335  0.1808  0.1197
[torch.DoubleTensor of size 2x10]



In [6]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

...Utils file loaded	


In [7]:
aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'
query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'

data_file = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})
query_file =  csvigo.load({path = query_fn, mode = "large"})

rK = 100
nbatches = 50
nepochs = 100
print_every = 10
embed_dim = 10
learning_rate = 0.1

cuts = 4.                  --- This is the number of cuts we want
epsilon = 1.
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) --- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs

cuda = true
torch.manualSeed(420)

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv	
<csv>	parsing done	


In [8]:
function build_network(vocab_size, embed_dim)
    model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.ReLU())
   return model
end
function build_model(vocab_size, embed_dim, outputSize)
    lstm1 = build_network(vocab_size, embed_dim)
    lstm2 = build_network(vocab_size, embed_dim)
    lstm3 = build_network(vocab_size, embed_dim)

    mlp1 = nn.Sequential()
    mlp1:add(nn.Linear(1, embed_dim))
    mlp1:add(nn.ReLU())

    ParallelModel = nn.ParallelTable()
    ParallelModel:add(lstm1)
    ParallelModel:add(lstm2)
    ParallelModel:add(lstm3)
    ParallelModel:add(mlp1)

    FinalMLP = nn.Sequential()
    FinalMLP:add(ParallelModel)
    FinalMLP:add(nn.JoinTable(2))
    FinalMLP:add( nn.Linear(embed_dim * 4, outputSize) )
    return FinalMLP
end

In [9]:
vocab_sized = getVocabSize(data_file)                       --- getting length of dictionary
vocab_sizeq = getVocabSize(query_file)                      --- getting length of dictionary
vocab_size = math.max(vocab_sized, vocab_sizeq)

queries = grabNsamples(query_file, #query_file-1, nil)      --- Extracting all queries
nuggets = grabNsamples(nugget_file, #nugget_file-1, nil)    --- Extracting all samples
maxlend = getMaxseq(data_file)                             --- Extracting maximum sequence length
maxlenq = getMaxseq(query_file)                            --- Extracting maximum sequence length
maxlen = math.max(maxlenq, maxlend)

batchLSTM = build_model(vocab_size, embed_dim, 1)
crit = nn.MSECriterion()

In [10]:
dofile("model_utils.lua")

In [ ]:
out = iterateModel( nbatches, nepochs, queries[3], data_file, batchLSTM, crit, epsilon, delta, 
                    maxlen, base_explore_rate, print_every, nuggets, learning_rate, rK)

In [11]:
rscores, pscores, fscores = {}, {}, {}

In [12]:
x = data_file
K = rK

In [13]:
preds_list = torch.totable(torch.round(torch.rand(#x)))
yrouge = torch.totable(torch.randn(#x))
summary_list = populateOnes(#x, K)
action_list = torch.totable(torch.round(torch.randn(#x)))

In [14]:
qs = queries[3]

In [15]:
nstart = 2
nend = 20

In [16]:
x_ss  = geti_n(x, nstart, nend)
xout  = grabNsamples(x_ss, 1, #x_ss)
xs  = padZeros(xout, maxlen)
qs2 = padZeros({qs}, 5)
qrep = repeatQuery(qs2[1], #xs)

In [17]:
preds = geti_n(preds_list, nstart, nend)

In [18]:
sumry_ss = buildPredSummary(preds, xs)

In [19]:
sumry_ss2 = padZeros(sumry_ss, maxlen)

In [20]:
yr_ss = geti_n(yrouge, nstart, nend)
as_ss = geti_n(action_list, nstart, nend)
sentences = torch.LongTensor(xs):t()
summary = torch.LongTensor(sumry_ss2):t()
query = torch.LongTensor(qrep):t()
actions = torch.Tensor(as_ss):resize(#xs, 1)
labels = torch.Tensor(yr_ss)

In [21]:
model = batchLSTM

In [ ]:
myPreds = model:forward({sentences, summary, query, actions})

In [ ]:
loss = loss + crit:forward(myPreds, labels)
grads = crit:backward(myPreds, labels)
model:backward({sentences, summary, query, actions}, grads)
model:updateParameters(learning_rate)        -- Update parameters after each minibatch
model:zeroGradParameters()

In [23]:
preds = policy(myPreds, epsilon, #xs)
--- Concatenating predictions into a summary
predsummary = buildPredSummary(preds, xs)

In [42]:
function getLastK(x, K)
    out = {}
    k = 0
    for i=0, #x-1 do
        if sumTable(x[#x - i])  ~= 0 then
            if k <= K-1 then
                --- 10 - 0 - 1 = 9..., 8, 7 ..., 1
                out[#x-i-1] = x[#x - i]
                k = k + 1
            else
                return out
            end
        end
    end
    return out
end

In [55]:
x = {
        {0, 1, 3, 4, 5}, 
        {1, 2, 3, 4, 5}, 
        {0, 0, 3, 4, 0}, 
        {0, 3, 1, 2, 5},
        {1, 1, 1, 1, 1}
}

In [58]:
geti_n(x, 2, 4)

{
  1 : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
    }
  2 : 
    {
      1 : 0
      2 : 0
      3 : 3
      4 : 4
      5 : 0
    }
  3 : 
    {
      1 : 0
      2 : 3
      3 : 1
      4 : 2
      5 : 5
    }
}


In [60]:
getLastK(geti_n(x, 2, 4), 2)

{
  2 : 
    {
      1 : 0
      2 : 3
      3 : 1
      4 : 2
      5 : 5
    }
}


In [48]:
getLastK(x, 3)

{
  1 : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
    }
  2 : 
    {
      1 : 0
      2 : 0
      3 : 3
      4 : 4
      5 : 0
    }
  3 : 
    {
      1 : 0
      2 : 3
      3 : 1
      4 : 2
      5 : 5
    }
}


In [ ]:
--- Initializing rouge metrics at time {t-1} and save scores
for i=1, #predsummary do
    --- Calculating rouge scores; Call get_i_n() to cumulatively compute rouge
    rscores[i] = rougeRecall(geti_n(predsummary, 1, i), nuggets, K) - r_t1
    pscores[i] = rougePrecision(geti_n(predsummary, 1, i), nuggets, K) - p_t1
    fscores[i] = rougeF1(geti_n(predsummary, 1, i), nuggets, K) - f_t1
    r_t1, p_t1, f_t1 = rscores[i], pscores[i], fscores[i]
end
--- Updating change in rouge
yrouge = updateTable(yrouge, fscores, nstart)
preds_list = updateTable(preds_list, preds, nstart)
--- Calculating last one to see actual last rouge, without delta
rscore = rougeRecall(predsummary, nuggets, K)
pscore = rougePrecision(predsummary, nuggets, K)
fscore = rougeF1(predsummary, nuggets, K)
